<a href="https://colab.research.google.com/github/alex-addanki/Experiments/blob/master/split_and_label_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas
from sklearn import model_selection
from sklearn.linear_model import LogisticRegression
import pickle
url = "https://raw.githubusercontent.com/jbrownlee/Datasets/master/pima-indians-diabetes.data.csv"
names = ['preg', 'plas', 'pres', 'skin', 'test', 'mass', 'pedi', 'age', 'class']
dataframe = pandas.read_csv(url, names=names)
array = dataframe.values
X = array[:,0:8]
Y = array[:,8]
test_size = 0.33
seed = 7
X_train, X_test, Y_train, Y_test = model_selection.train_test_split(X, Y, test_size=test_size, random_state=seed)


In [2]:
# Fit the model on 33%
model = LogisticRegression()
model.fit(X_train, Y_train)
# save the model to disk


/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [0]:
filename = 'finalized_model.sav'
pickle.dump(model, open(filename, 'wb'))

In [4]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(X_test, Y_test)
print(result)

0.7559055118110236


In [7]:
from IPython import get_ipython
ipython = get_ipython()
ipython.magic("sx unzip recordings.zip")

['Archive:  recordings.zip', '   creating: recordings/']

In [10]:
import os
from collections import defaultdict

import numpy as np
from scipy.io.wavfile import read, write

from say_numbers_prompt import generate_number_sequence, DELAY_BETWEEN_NUMBERS

"""
Splits up the audio data you collected in Audacity.

Adjust the CONSTANTS below and run this file.

Labeled audio will appear in the "recordings" dir.
"""

YOUR_NAME_HERE = 'Alex'

# Where did you save your Audacity-exported wav file?
PATH_TO_AUDIO_FILE = r'Test-2.wav'

# Time (seconds) between the beginning of the file and the first number
# If your output files end up silent, change this number!
# It may help to look at the beginning of your recording in Audacity to see the offset.
START_OFFSET = 6

# How long it actually took you to say each number, typically 1.5 seconds
SECS_PER_NUMBER = 3

LABELS = generate_number_sequence()


def split_wav(start_offset, secs_between_numbers, secs_per_number):
    fname = PATH_TO_AUDIO_FILE
    rate, sound = read(fname)

    if len(sound.shape) > 1:
        # Audio probably has L and R channels.
        # Use the left channel only (mono).
        sound = sound[:, 0]

    samples_between_numbers = int(rate * secs_between_numbers)
    offset_idx = int(rate*start_offset)

    counts = defaultdict(lambda: 0)

    for i, label in enumerate(LABELS):
        label = str(label)
        start_idx = offset_idx + i * samples_between_numbers
        stop_idx = start_idx + int(rate * secs_per_number)

        if stop_idx > len(sound):
            raise('Error: Sound ends before expected number of samples reached for index:' + str(i))

        # trim silence
        digit_audio = sound[start_idx:stop_idx]
        digit_audio_trimmed = trim_silence(digit_audio)

        # Build filename
        outfile = label + "_" + YOUR_NAME_HERE + "_" + str(counts[label]) + ".wav"
        outfile = 'recordings' + os.sep + outfile

        # Write audio chunk to file
        print ("writing", outfile)
        write(outfile, rate, digit_audio_trimmed)
        counts[label] += 1



def trim_silence(audio, n_noise_samples=1000, noise_factor=1.0, mean_filter_size=100):
    """ Removes the silence at the beginning and end of the passed audio data
    Fits noise based on the last n_noise_samples samples in the period
    Finds where the mean-filtered magnitude > noise
    :param audio: numpy array of audio
    :return: a trimmed numpy array
    """
    start = 0
    end = len(audio)-1

    mag = abs(audio)

    noise_sample_period = mag[end-n_noise_samples:end]
    noise_threshold = noise_sample_period.max()*noise_factor

    mag_mean = np.convolve(mag, [1/float(mean_filter_size)]*mean_filter_size, 'same')

    # find onset
    for idx, point in enumerate(mag_mean):
        if point > noise_threshold:
            start = idx
            break

    # Reverse the array for trimming the end
    for idx, point in enumerate(mag_mean[::-1]):
        if point > noise_threshold:
            end = len(audio) - idx
            break

    return audio[start:end]


if __name__ == '__main__':
    split_wav(START_OFFSET, DELAY_BETWEEN_NUMBERS, SECS_PER_NUMBER)



writing recordings/0_Alex_0.wav
writing recordings/9_Alex_0.wav
writing recordings/2_Alex_0.wav
writing recordings/1_Alex_0.wav
writing recordings/4_Alex_0.wav
writing recordings/3_Alex_0.wav
writing recordings/7_Alex_0.wav
writing recordings/7_Alex_1.wav
writing recordings/4_Alex_1.wav
writing recordings/2_Alex_1.wav
writing recordings/1_Alex_1.wav
writing recordings/1_Alex_2.wav
writing recordings/5_Alex_0.wav
writing recordings/5_Alex_1.wav
writing recordings/4_Alex_2.wav
writing recordings/8_Alex_0.wav
writing recordings/6_Alex_0.wav
writing recordings/7_Alex_2.wav
writing recordings/9_Alex_1.wav
writing recordings/2_Alex_2.wav
writing recordings/3_Alex_1.wav
writing recordings/6_Alex_1.wav
writing recordings/8_Alex_1.wav
writing recordings/8_Alex_2.wav
writing recordings/5_Alex_2.wav
writing recordings/6_Alex_2.wav
writing recordings/9_Alex_2.wav
writing recordings/0_Alex_1.wav


TypeError: ignored